# TensorFlow - MNIST For ML Beginners - Next Step: FCNN (Part II) 

This Code Along tutorial is an extension of TensorFlow.org's MNIST For ML Beginners tutorial. They describe this as the Hello World example for those just starting to use TensorFlow. In this tutorial, we extend it from a two-layer softmax regression neural network to a three layer fully-connected neural network (FCNN) with a hidden layer.

In [ ]:
# Importing Tensorflow
import tensorflow as tf
from tensorflow.python.framework import ops

## What is MNIST

The MNIST dataset is a well-known dataset of images of handwritten digits (0 .. 9). It is used extensively as an example for
training beginners in both Computer Vision and ML frameworks. 

Kaggle describes this dataset as:

MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

## Images

<img src='mnist.png'>

Each image has a label, between 0 and 9 and represents a single digit. Each image is 28 pixels by 28 pixels. The pixels are black or white (0 or 1). Each image represents a total of 784 pixels (28 x 28)

## Getting the Data

Tensorflow comes with several builtin datasets to get started. These are located in the package tensorflow.examples.tutorials.

The dataset contains a total of 70,000 images, split up as follows:

- 55,000 training data
- 10,000 test data
- 5,000 validation data

Let's start by getting the data from tensorflow.

In [ ]:
# Import the MNIST input_data function from the tutorials.mnist package
from tensorflow.examples.tutorials.mnist import input_data

# Read in the data
# The paramter one_hot=True refers to the label which is categorical (1-10). 
# The paramter causes the label to be re-encoded as a 10 column vector.
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Let's now look at what type of object 'mnist' is:

In [ ]:
type(mnist)

You can see that it is a Tensorflow object of type Datasets. The training, test and validation data can be accessed
as attributes. 

Let's look at their data types. We can see they are Tensorflow Dataset (non-plural) data types.

In [ ]:
print( type(mnist.train) )
print( type(mnist.test) )
print( type(mnist.validation) )

## Inside the Dataset

Let's now look closer to what's inside the Tensorflow dataset object for the MNIST dataset. 

The images and corresponding labels are accessed by the attributes images and labels, respectively. How convenient, such a logical name for an attribute! 

Both the images and labels are a Numpy multi-dimensional array.

In [ ]:
type(mnist.train.images)

Let's check that the training data has the expected 55,000 images and labels.

In [ ]:
# Let's get the length (number of images) of the list of images.
print( len(mnist.train.images) )
print( len(mnist.train.labels) )

Let's now look at the contents of one of the images.

In [ ]:
mnist.train.images[0]


The original black and white (bilevel) images from MNIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. The images were centered in a 28x28 image by computing the center of mass of the pixels, and translating the image so as to position this point at the center of the 28x28 field.  - http://yann.lecun.com/exdb/mnist/

### Plotting an Image

Let's plot one of the images in the training set. To do so, we will use the plotting functions of the matplotlib package. Let's start by importing the package.

In [ ]:
import matplotlib.pyplot as plt

# This line is specific to python notebooks (not python). 
# It causes plots to automatically be rendered (displayed) without issuing a show command.
%matplotlib inline  

Let's now render (display) one of the images. 

<b>WAIT</b>, the images in the Tensorflow MNIST dataset are already flattened. That is, they are a single vector of 784 inputs. To display it, we need to reshape the vector back into a 28 x 28 pixel matrix. Then we can plot it.

In [ ]:
# Let's show that the shape of the image is already flatten (will output as 784)
print( mnist.train.images[1].shape )

# Let's now reshape it into a 28 x 28 matrix
image = mnist.train.images[1].reshape(28,28)
print( image.shape )

# Let's plot it now
plt.imshow( image )

### The Labels

The corresponding labels are a 2D matrix. Each row is for the corresponding image (that is index 0 in labels is for image at index 0 in images).

Each row has ten columns. The labels have been encoded as a one-hot encoding. In this case, if the image is a 3, then there is a 1 at index 3, and all the other columns are a zero.

Let's look at the label vector for the image above.

In [ ]:
# As you can see, there is a 1 at index 3 (fourth location, starting at 0)
mnist.train.labels[1]

## Fully-Connected Neural Network - Softmax Activation

We are going to build a simple fully-connected neural network (FCNN). There will not be a convolutional layer. That is, we will use all 784 pixels as input. The output layer from our neural network will be passed through a softmax activation function to produce our predictions of the digit.

In our Neural Network, we will have the following:

    - An input layer of 784 inputs and 64 outputs
    - A linear recitifier activation function
    - A hidden layer of 64 inputs and 20 outputs
    - A linear recitifier activation function
    - An output layer of 20 inputs and 10 outputs
    - A softmax activation function

INPUT LAYER => RELU => HIDDEN LAYER => RELU => OUTPUT LAYER => SOFTMAX

### Softmax

We will use the softmax layer to make our predictions. Each output from softmax will be a number between 0 and 1, representing a percent. That is, if the output for the node 3 is 0.8, then this means 80% prediction. We will choose the output with the highest percent when making a prediction.

Softmax is a mathematical function that takes a set of values, which may otherwise not add up to 1, and outputs a new set of numbers when totaled will add up to 1. That is, we use softmax() so that all our outputs for each image add up to 1 (100%).

Softmax will be our 'activation' function from the output layer.

## Building the Neural Network

Let's build our neural network now. We will do the following:

### Design, then Run

#### Design

    - Create the placeholder for the input data
    - Design the layers
    - Set the optimizer
    
#### Run

      -- Initialize the Graph
      -- Set number of epochs
      -- Set batch size, learning rate
      -- Run the Graph with the Training Data to Train (learn) the Model
      -- Validate the Model with Test Data

### Input Vector and Output Vector Placeholders

For our first tensorflow step, we will setup the Tensorflow placeholders.

We have two placeholders we need to declare, one for the input vector (pixel image data) and one for the output vector (digit classifier).

For our input placeholder (which we call X), we have 784 features (pixels per image). For the output vector (which we call Y), we have have 10 classifiers (0 .. 9 digits). In both cases, we set the second dimension of our vector to None. The None is
a placeholder for the number of samples we will feed into the neural network at run-time. We also know that our data is floating point values between 0 and 1, so we will set the data type to float32.

In [ ]:
# Let's first reset our graph, so our neural network components are all declared within the same graph
ops.reset_default_graph() 

In [ ]:
X = tf.placeholder(tf.float32, [784, None])
Y = tf.placeholder(tf.float32, [10, None])

### INPUT LAYER

Let's now design our input layer. We need two things: weights and biases. 

Each input feature (pixel) will need a weight (which our model will learn during training). The weight is multipled against the value of the input (pixel), which we symbolically represent as Wx. 

Each output from the layer will need a bias (which our model will learn during training). The bias is added to the result of the weight multipled by the pixel value (Wx).

Let's create two tensorflow variables for our weights and biases. The weights (which we call W) will need to be a 2D matrix. The rows are the number of inputs, which is 784 and the columns the number of outputs to the hidden layer, which will be 64.

The bias will be a vector of size 64 (one for each output).

We need to initialize our weights and biases to some initial value. We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
tf.set_random_seed(1)   # Set the same seed to get the same initialization as in this demo.

# The weights for the input layer
W1 = tf.get_variable("W1", [64, 784], initializer=tf.contrib.layers.xavier_initializer(seed=1))

# The bias for the output from the input layer
b1 = tf.get_variable("b1", [64, 1], initializer=tf.zeros_initializer())

Let's put it together into an input layer. We will use the Tensorflow method tf.matmul() to do a matrix multiplication of the weights (our variable W1) and the inputs (our placeholder X), add in the bias (b1), and pass the output through a linear activation function.

- Create a node that will multiply the weights (W1) against the input vector (X - which is our input placeholder).
- Create a node that adds the bias to the above node (W1 * X)
- Pass the outputs from the input layer through a RELU activation function

In [ ]:
# The first layer (input layer)
Z1 = tf.add(tf.matmul(W1, X), b1)

# Let's add the activation function to the output signal from the first layer
A1 = tf.nn.relu(Z1)

### HIDDEN LAYER

The hidden layer will have 64 inputs (outputs from input layer) and 20 outputs. Each input will need a weight and each output a bias (which we will train). Each output will be passed through the linear rectifier unit (RELU) activation function.

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
W2 = tf.get_variable("W2", [20, 64], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b2 = tf.get_variable("b2", [20, 1], initializer=tf.zeros_initializer())

Let's construct the hidden layer

- Create a node that will multiply the weights (W2) against the outputs of the input layer (A1).
- Create a node that adds the bias to the above node (W2 * A1)
- Pass the outputs from the (first) hidden layer through a RELU activation function

In [ ]:
# The second layer (hidden layer)
Z2 = tf.add(tf.matmul(W2, A1), b2) 

# Let's add the activation function to the output signal from the second layer
A2 = tf.nn.relu(Z2)

### OUTPUT LAYER

The output layer will have 20 inputs (outputs from the hidden layer) and 10 outputs (one for each digit). Each input will need a weight and each output a bias (which we will train). The 10 outputs will be passed through a softmax activation function. 

We will initialize the weights using a random value initializer (Xavier) and initialize the biases to zero.

In [ ]:
W3 = tf.get_variable("W3", [10, 20], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b3 = tf.get_variable("b3", [10, 1], initializer=tf.zeros_initializer())

Let's construct the output layer

- Create a node that will multiply the weights (W3) against the outputs of the hidden layer (A2).
- Create a node that adds the bias to the above node (W3 * A2)
- Pass the outputs from the output layer through a SOFTMAX squashing function (done by the optimizer)

In [ ]:
# The third layer (outout layer)
Z3 = tf.add(tf.matmul(W3, A2), b3) 

## OPTIMIZER

Now its time to design our optimizer. Let's start by designing our cost function. We will use the mean value of the softmax cross entropy between the predicted labels and actual labels. This is what we want to reduce on each batch.

In [ ]:
# to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=tf.transpose(Z3), labels=tf.transpose(Y)))

Let's design our optimizer. This is the method that adjusts the values of the weights and biases, based on minizing the cost value during training.

We also need to set a learning rate. This is multiplied against the gradient calculation. It's used to prevent huge swings in setting weights which can result in either converging at a local (instead of global) optima, or not converging at all (infinite gradient).

In [ ]:
# The learning rate for Gradient Descent algorithm
learning_rate = 0.5

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

### Run the Graph

We've built our Tensorflow graph for training our data. So, let's start training it.

First, we need to call Tensorflow's global_variables_initializer() method to initialize the variables we've defined. We will create this as another node, which will be the first node we run (evaluate) in our graph.

In [ ]:
init = tf.global_variables_initializer()

It's also a good idea to know how long your training takes, so let's import the time library.

In [ ]:
import time

Let's set our hyperparameters.

We need to set the number of epochs (that's how many times we run the training data through the neural network), and the batch size. The batch size is a small subset of the entire training set. We will be running a batch at a time per epoch. After each batch, then the cost is computed and backpropagated through the neural network.

In [ ]:
epochs = 1000      # run a 1000 epochs
batch_size = 200   # for each epoch, train in batches of 200 images

We are going to run the graph now!

We start by creating a tensorflow session (tf.Session()). Within the session we can run (evaluate) parts of the graph we designed.

We start by initializing the tensor variables we defined for the weights and biases.

We then run our training data through our neural network for the number of epochs we defined. For each epoch, we get a randomly shuffled batch from the training data and feed the batch (i.e. feed dictionary) into the neural network by running (evaluate)
the optimizer node in our graph.

Once we've trained the model, then we create some new nodes to calculate accuracy and evaluate against the training data.

In [ ]:
start = time.time()

with tf.Session() as sess:
    # Initialize the variables
    sess.run(init)
    
    # run our training data through the neural network for each epoch
    for epoch in range(epochs):
      # Get a batch (random shuffled) from the training data
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      
      # Feed this batch through the neural network.
      _, epoch_cost = sess.run([optimizer, cost], feed_dict={X: batch_xs.T, Y: batch_ys.T})
      
      if epoch % 100 == 0:
        print("Epoch: ", epoch, epoch_cost)
        
    end = time.time()
    
    print("Training Time:", end - start)
    
    # Test the Model
    
    # Let's select the highest percent from the softmax output per image as the prediction.
    prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))
    
    # Let's create another node for calculating the accuracy
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

    # Now let's run our trainingt images through the model to calculate our accuracy during training
    print ("Train Accuracy:", accuracy.eval({X: mnist.train.images.T, Y: mnist.train.labels.T}))
    
    # Now let's run our test images through the model to calculate our accuracy on the test data
    print ("Test Accuracy:", accuracy.eval({X: mnist.test.images.T, Y: mnist.test.labels.T}))

### Evaluate of Model

The last three steps above is where our test data was ran through the model and produced how accurate our model was on the test data.

After training the model, we created a node for prediction. This node compares two vectors, our predicted labels and our actual labels. Each vector is 10 elements long with a 1 in the predicted/actual digit location. So we are comparing the vectors. If they match (prediction matches actual), then we have a TRUE; otherwise a FALSE. That's how we are going to get our accuracy percentage calculated.

Next, we create the node accuracy. This node is a cost function!

We then run the accuracy node, feeding it the test images as the X variable and the test labels as the Y variable. This will result in the test images being ran through the model (which is in memory) and the corresponding output vectors evaluated against the actual labels of the test images (Y).

## You Have Now Completed TensorFlow's MNIST Example Extended to a FCNN

As you can see, by adding a single hidden layer we have increased the accuracy on the test data from 72% to 96%